In [1]:
import numpy as np
import math
import matplotlib as mpl
import random

$$dr_t = \alpha (\theta_t - r_t)dt + \sigma dw_t$$
$$r_{t+1} = r_t + dr_{t}$$

In [2]:
N = 10
M = 10000
alpha = 0.1
sigma = 0.15
corr = 0.1

In [3]:
yld_UST = [0.0312, 0.0320, 0.0325, 0.0328, 0.0333, 0.0337, 0.0340, 0.0343, 0.0345, 0.0347]
yld_TED = [0.0045, 0.0094, 0.0084, 0.0095, 0.0092, 0.0089, 0.0087, 0.0085, 0.0084, 0.0083]
yld_LIB = yld_UST + yld_TED

vol_UST = [0.15, 0.15, 0.15, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]
vol_TED = [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05]

In [5]:
# set seed
random.seed(46977)

# create brownian motions
BM_UST = [np.random.standard_normal(N) for i in range(M)]
BM_LIB = corr * BM_UST + sqrt(1 - corr**2) * [np.random.standard_normal(N) for i in range(M)]

# 

TypeError: can't multiply sequence by non-int of type 'float'

In [ ]:
def sum_discount_factor(t, monte_carlo_paths, BM_paths):
    result = 0
    for i in range (M):
        IR_path = monte_carlo_paths[i]
        BM_path = BM_paths[i]
        result += math.exp(-sum(IR_path[0:t]) - (2-alpha) * IR_path[t] + sigma * IR_path[t] * BM_path[t] )
    return result

In [ ]:
def monte_carlo(yield_curve, BM_paths):
    thetas = np.zeros(N-1)
    monte_carlo_paths = [np.zeros(N) for i in range(M)]
    # spot rate(0,1) = yield(0,1)
    for path in monte_carlo_paths:
        path[0] = yield_curve[0]

    for t in range(N-1):
        thetas[t] = -1/alpha * ( math.log(M) - (t+2) * yield_curve[t+1] - math.log( sum_discount_factor( t, monte_carlo_paths, BM_paths ) ) )
        for i in range (M):
            IR_path = monte_carlo_paths[i]
            BM_path = BM_paths[i]
            IR_path[t+1] = IR_path[t] + alpha * ( thetas[t] - IR_path[t] ) + sigma * IR_path[t] * BM_path[t]
            monte_carlo_paths[i] = IR_path
    return (monte_carlo_paths, thetas)
        

In [ ]:
(monte_carlo_paths, thetas) = monte_carlo(yld_UST, BM_UST)
print(monte_carlo_paths)
print(thetas)

In [ ]:
def sanity_check(monte_carlo_paths, yield_curve):
    real_discount_factors = np.zeros(N)
    learned_discount_factors = np.zeros(N)
    diffs = np.zeros(N)
    for t in range (N):
        real_discount_factor = math.exp( -( t+1 ) * yield_curve[t] )
        real_discount_factors[t] = real_discount_factor
        path_discount_factors = np.zeros(M)
        for i in range (M):
            IR_path = monte_carlo_paths[i]
            path_discount_factors[i] = math.exp(-sum(IR_path[0:t+1]))
        learned_discount_factors[t] = np.mean(path_discount_factors)
        diff = learned_discount_factors[t] - real_discount_factor
        diffs[t] = diff
    return ( real_discount_factors, learned_discount_factors, diffs )

In [ ]:
( real_discount_factors, learned_discount_factors, diff ) = sanity_check( monte_carlo_paths, yld_UST )
print( real_discount_factors )
print( learned_discount_factors )
print( diff )